# Cliente API 5G Data Generator

Este notebook interactúa con la API para:
1. Generar datasets
2. Consultar datos de MongoDB
3. Generar datos sintéticos usando RAG

In [1]:
import requests
import pandas as pd
import json
from IPython.display import display

BASE_URL = "http://localhost:8000"

## 1. Generar Dataset Base

In [2]:
def generar_dataset(num_samples=1000):
    """Genera un nuevo dataset usando la API"""
    try:
        response = requests.post(f"{BASE_URL}/generate", 
                               params={"num_samples": num_samples})
        response.raise_for_status()
        data = response.json()
        
        print(f"Dataset ID: {data['dataset_id']}")
        print(f"Número de muestras: {num_samples}")
        
        return pd.DataFrame(data['preview']), data['dataset_id']
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Generar dataset
df_preview, dataset_id = generar_dataset(num_samples=1000)
if df_preview is not None:
    print("\nPreview del dataset:")
    display(df_preview)

Dataset ID: 54609087-d038-4272-8300-33f248c88a27
Número de muestras: 1000

Preview del dataset:


,device_id,timestamp,bandwidth_mbps,latency_ms,packet_loss,signal_strength_dbm,cell_id,connection_type
0,5128,2024-12-15T12:38:38.799145,682.735734,7.009634,0.030280,-85.833446,18,Beamforming
1,4724,2024-12-15T13:38:38.799153,240.026772,11.252104,0.097497,-120.000000,63,Carrier Aggregation
2,2620,2024-12-15T14:38:38.799154,165.625063,12.307276,0.084711,-107.185166,55,Carrier Aggregation
3,1915,2024-12-15T15:38:38.799155,487.885106,16.029175,0.023890,-98.241309,44,MIMO
4,6999,2024-12-15T16:38:38.799156,738.140944,5.577651,0.119253,-103.009525,96,MIMO


## 2. Consultar Datos de MongoDB

In [3]:
def consultar_mongodb(dataset_id=None, limit=1000, format="json"):
    """Consulta datos de MongoDB"""
    try:
        params = {
            'limit': limit,
            'format': format
        }
        if dataset_id:
            params['dataset_id'] = dataset_id
            
        response = requests.get(f"{BASE_URL}/get-mongodb-data", params=params)
        response.raise_for_status()
        
        if format == 'csv':
            # Guardar CSV
            with open('datos_mongodb.csv', 'wb') as f:
                f.write(response.content)
            print("Datos guardados en 'datos_mongodb.csv'")
            return pd.read_csv('datos_mongodb.csv')
        else:
            data = response.json()
            print(f"Total registros: {data['total_records']}")
            return pd.DataFrame(data['data'])
            
    except Exception as e:
        print(f"Error: {e}")
        return None

# Consultar datos usando el dataset_id generado
if dataset_id:
    print("\nConsultando datos del dataset generado:")
    df_mongo = consultar_mongodb(dataset_id=dataset_id)
    if df_mongo is not None:
        display(df_mongo.head())
        display(df_mongo.describe())


Consultando datos del dataset generado:
Total registros: 1000


,device_id,timestamp,bandwidth_mbps,latency_ms,packet_loss,signal_strength_dbm,cell_id,connection_type,dataset_id
0,5128,2024-12-15T12:38:38.799145,682.735734,7.009634,0.030280,-85.833446,18,Beamforming,54609087-d038-4272-8300-33f248c88a27
1,4724,2024-12-15T13:38:38.799153,240.026772,11.252104,0.097497,-120.000000,63,Carrier Aggregation,54609087-d038-4272-8300-33f248c88a27
2,2620,2024-12-15T14:38:38.799154,165.625063,12.307276,0.084711,-107.185166,55,Carrier Aggregation,54609087-d038-4272-8300-33f248c88a27
3,1915,2024-12-15T15:38:38.799155,487.885106,16.029175,0.023890,-98.241309,44,MIMO,54609087-d038-4272-8300-33f248c88a27
4,6999,2024-12-15T16:38:38.799156,738.140944,5.577651,0.119253,-103.009525,96,MIMO,54609087-d038-4272-8300-33f248c88a27


,device_id,bandwidth_mbps,latency_ms,packet_loss,signal_strength_dbm,cell_id
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5731.666000,505.078669,9.943276,0.038959,-94.879087,50.717000
std,2641.370187,152.184771,3.111029,0.027325,9.733667,28.189458
min,1013.000000,0.000000,0.000000,0.001315,-120.000000,1.000000
25%,3312.500000,395.820600,7.813619,0.018495,-101.466149,27.000000
50%,5884.000000,497.506152,9.918280,0.033341,-94.870560,51.000000
75%,8111.500000,609.364766,12.155511,0.051411,-88.110749,75.000000
max,9998.000000,984.806482,19.588298,0.172016,-70.000000,99.000000


## 3. Generar Datos con RAG desde JSON

In [3]:
def generar_desde_json(json_file, num_samples=2, context_query="Analiza los patrones y genera datos similares"):
    """Genera datos usando RAG desde un archivo JSON"""
    try:
        files = {'file': ('data.json', open(json_file, 'rb'))}
        params = {
            'num_samples': num_samples,
            'context_query': context_query
        }
        
        response = requests.post(
            f"{BASE_URL}/ollama-data-generate-from-json",
            files=files,
            params=params
        )
        
        # Guardar CSV generado
        with open('data_json_generated.csv', 'wb') as f:
            f.write(response.content)
        
        print("Datos guardados en 'data_json_generated.csv'")
        return pd.read_csv('data_json_generated.csv')
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Ejemplo de uso con un archivo JSON
json_file = 'json_dataset.json'  # Asegúrate de tener este archivo
df_rag_json = generar_desde_json(
    json_file,
    num_samples=5,
    context_query="Genera datos manteniendo correlaciones entre campos pero con valores diferentes"
)

if df_rag_json is not None:
    print("\nDatos generados desde JSON:")
    display(df_rag_json)

Datos guardados en 'data_json_generated.csv'

Datos generados desde JSON:


,device_id,timestamp,bandwidth_mbps,latency_ms,packet_loss,signal_strength_dbm,cell_id,connection_type
0,6342,2024-12-28T15:46:24.173990,523.812945,10.345892,0.031745,-98.675432,37,Carrier Aggregation
1,6347,2025-02-15T18:46:24.173987,523.456789,10.234567,0.031234,-98.765432,37,Carrier Aggregation
2,6342,2024-12-28T15:46:24.173987,523.456789,10.234567,0.031234,-98.765432,42,Carrier Aggregation
3,6342,2025-02-15T18:46:24.173987,520.740000,10.340000,0.030000,-98.560000,35,Carrier Aggregation
4,6347,2025-02-15T18:46:24.173987,523.123457,10.234568,0.031235,-98.765432,37,Carrier Aggregation


## 4. Generar Datos con RAG desde CSV

In [5]:
def generar_desde_csv(csv_file, num_samples=5, context_query="Analiza los patrones y genera datos similares"):
    """Genera datos usando RAG desde un archivo CSV"""
    try:
        files = {'file': ('data.csv', open(csv_file, 'rb'))}
        params = {
            'num_samples': num_samples,
            'context_query': context_query
        }
        
        response = requests.post(
            f"{BASE_URL}/ollama-data-generate-from-csv",
            files=files,
            params=params
        )
        
        # Guardar CSV generado
        with open('data_csv_generated.csv', 'wb') as f:
            f.write(response.content)
        
        print("Datos guardados en 'datos_generados_csv.csv'")
        return pd.read_csv('data_csv_generated.csv')
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Ejemplo de uso con un archivo CSV
csv_file = 'csv_dataset.csv'  # Asegúrate de tener este archivo
df_rag_csv = generar_desde_csv(
    csv_file,
    num_samples=5,
    context_query="Analiza los patrones y genera datos similares"
)

if df_rag_csv is not None:
    print("\nDatos generados desde CSV:")
    display(df_rag_csv)

## 5. Comparar Estadísticas

In [ ]:
def comparar_estadisticas(df1, df2, nombre1="Original", nombre2="Generado"):
    """Compara estadísticas entre dos DataFrames"""
    if df1 is not None and df2 is not None:
        print(f"\nEstadísticas {nombre1}:")
        display(df1.describe())
        
        print(f"\nEstadísticas {nombre2}:")
        display(df2.describe())

# Comparar datos originales con generados
if df_mongo is not None and df_rag_json is not None:
    comparar_estadisticas(df_mongo, df_rag_json, "MongoDB", "RAG JSON")

if df_mongo is not None and df_rag_csv is not None:
    comparar_estadisticas(df_mongo, df_rag_csv, "MongoDB", "RAG CSV")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from datetime import datetime

def analizar_datasets(df_original, df_generado, nombre_original, nombre_generado):
    """
    Analiza y compara dos datasets, mostrando estadísticas, gráficos y métricas
    """
    # 1. Tiempo de inicio
    tiempo_inicio = time.time()
    
    # 2. Comparación de las primeras 5 columnas
    columnas_numericas = df_original.select_dtypes(include=['float64', 'int64']).columns[:5]
    
    # 3. Crear figura para múltiples gráficos
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Comparación de Datasets: Original vs Generado', fontsize=16)
    
    # 4. Comparar y graficar cada columna
    for idx, col in enumerate(columnas_numericas):
        ax = axes[idx // 3, idx % 3]
        
        # Boxplot comparativo
        data_to_plot = [
            df_original[col].values,
            df_generado[col].values
        ]
        
        sns.boxplot(data=data_to_plot, ax=ax)
        ax.set_xticklabels(['Original', 'Generado'])
        ax.set_title(f'Distribución de {col}')
        
        # Calcular diferencias estadísticas
        diff_mean = abs(df_original[col].mean() - df_generado[col].mean())
        diff_std = abs(df_original[col].std() - df_generado[col].std())
        
        print(f"\nDiferencias en {col}:")
        print(f"Diferencia en media: {diff_mean:.2f}")
        print(f"Diferencia en desviación estándar: {diff_std:.2f}")
    
    plt.tight_layout()
    plt.show()
    
    # 5. Calcular tamaños de archivo
    def get_file_size(filename):
        return os.path.getsize(filename) / 1024  # KB
    
    original_size = get_file_size(nombre_original)
    generado_size = get_file_size(nombre_generado)
    
    # 6. Calcular palabras por minuto (aproximado por número de celdas)
    tiempo_total = time.time() - tiempo_inicio
    celdas_por_minuto_original = (len(df_original) * len(df_original.columns)) / (tiempo_total/60)
    celdas_por_minuto_generado = (len(df_generado) * len(df_generado.columns)) / (tiempo_total/60)
    
    # 7. Mostrar resumen
    print("\n=== Resumen de Comparación ===")
    print(f"\nTamaños de Archivo:")
    print(f"Original: {original_size:.2f} KB")
    print(f"Generado: {generado_size:.2f} KB")
    
    print(f"\nTiempo de Procesamiento:")
    print(f"Total: {tiempo_total:.2f} segundos")
    
    print(f"\nVelocidad de Procesamiento:")
    print(f"Original: {celdas_por_minuto_original:.0f} celdas/minuto")
    print(f"Generado: {celdas_por_minuto_generado:.0f} celdas/minuto")
    
    print(f"\nModelo Utilizado:")
    print("Nombre: qwen2.5")
    print("Tipo: Ollama")
    
    # 8. Graficar comparación de tamaños
    plt.figure(figsize=(10, 6))
    plt.bar(['Original', 'Generado'], [original_size, generado_size])
    plt.title('Comparación de Tamaños de Archivo')
    plt.ylabel('Tamaño (KB)')
    plt.show()

# Ejecutar el análisis
if df_mongo is not None and df_rag_json is not None:
    print("Comparando Dataset Original vs Generado por RAG:")
    analizar_datasets(
        df_mongo,
        df_rag_json,
        'datos_mongodb.csv',
        'data_json_generated.csv'
    )